### Convert the tlp file output by GrouseFlocks to MyJSON 
* Observing that all of the tlp files are at most two level, this script only deals with two level
* We treat each open metanode as a "true metanode" that has underlying leaf nodes, but the closed metanode does not have underlying leaf nodes.  They are both in the data structure "metanodes"

In [1]:
from tulip import tlp
from shapely.geometry import *
import os
import math
import json
SQUARE_ROOT_2 = math.sqrt(2)

In [2]:
DATA_DIR = '../../../data/real-world-compiled/snap-main-comp/'
grousetlp_filepath = os.path.join(DATA_DIR, 'snap-email-eu-core-main-comp-grouseflocks-open-6.tlp')
myjson_filepath = os.path.join(DATA_DIR, 'snap-email-eu-core-main-comp-grouseflocks-open-6.json')

In [3]:
graph = tlp.loadGraph(grousetlp_filepath)
hierarchy_subgraphs = graph.getSubGraph('1__HierarchySubGraphs')
hierarchy_unnamed = hierarchy_subgraphs.getSubGraph('unnamed')
final_layout = graph.getSubGraph('1__Final Layout Display')
assert(hierarchy_subgraphs is not None)
assert(hierarchy_unnamed)
assert(final_layout is not None)

In [4]:
metanodes = {}
leaf_nodes = []
edges = []
root = graph.getId()

In [5]:
# Find the coodinates and sizes for all metanodes (both open and closed)
sub_view_layout = hierarchy_unnamed.getLayoutProperty('viewLayout')
sub_view_size = hierarchy_unnamed.getSizeProperty('viewSize')
view_hier_attr = hierarchy_unnamed.getIntegerProperty('view Hier Node to Display')
view_meta_graph = graph.getGraphProperty('viewMetaGraph')
parents = {}

for n in hierarchy_unnamed.getNodes():
    center_coor = sub_view_layout[n]
    # Use the diagonal of a bounding square as the diameter of bounding circle
    diameter = sub_view_size[n][0] * SQUARE_ROOT_2
    # print(n.id, center_coor, square_diagonal)
    metanodes[n.id] = {
        'id': n.id,   # Use the node id instead of the subgraph graph id for metanode.  be consistant
        'parent_metanode': root,
        'desc_metanodes': {},
        'level': 2,
        'leaf_nodes': {},
        'diameter': diameter,
        'geometry': Point(center_coor.x(), center_coor.y()).buffer(diameter / 2.0, cap_style=CAP_STYLE.round)
    }
    if view_hier_attr[n] == -1:
        # indicate this is an open metanode
        print('open metanode: ', n.id, view_meta_graph[n])
        for leaf in view_meta_graph[n].getNodes():
            metanodes[n.id]['leaf_nodes'][leaf.id] = True
            parents[leaf.id] = n.id
        
print(len(metanodes))
# print(metanodes.keys())

open metanode:  994 <graph "Contracted 6" (id 216) >
open metanode:  1000 <graph "Contracted 12" (id 222) >
open metanode:  1048 <graph "Contracted 60" (id 270) >
open metanode:  1132 <graph "Contracted 144" (id 354) >
open metanode:  1173 <graph "Contracted 185" (id 395) >
open metanode:  1176 <graph "Contracted 188" (id 398) >
200


In [6]:
# Find out all leaf nodes
view_hier_attr = final_layout.getIntegerProperty('view Hier Node to Display')
sub_view_layout = final_layout.getLayoutProperty('viewLayout')
sub_view_size = final_layout.getSizeProperty('viewSize')

for n in final_layout.getNodes():
    center_coor = sub_view_layout[n]
    width = sub_view_size[n][0]
    # print(n.id, center_coor, diameter)
    if view_hier_attr[n] != -1:
        # leaf nodes
        print('leaf: ', n.id)
        # Leaf node is only possible when a metanode is open
        assert(n.id in parents)      
        leaf_nodes.append({
            'id': n.id,
            'parent_metanode': parents[n.id], 
            'geometry': Point(center_coor.x(), center_coor.y()).buffer(width / 2.0, cap_style=CAP_STYLE.square),
            'diameter': width * SQUARE_ROOT_2,
        })       
print(len(leaf_nodes))

leaf:  8
leaf:  9
leaf:  15
leaf:  82
leaf:  141
leaf:  163
leaf:  216
leaf:  233
leaf:  239
leaf:  288
leaf:  334
leaf:  335
leaf:  405
leaf:  414
leaf:  415
leaf:  432
leaf:  449
leaf:  456
leaf:  543
leaf:  567
leaf:  579
leaf:  602
leaf:  604
leaf:  635
leaf:  639
leaf:  667
leaf:  679
leaf:  680
leaf:  687
leaf:  725
leaf:  740
leaf:  744
leaf:  756
leaf:  777
leaf:  779
leaf:  798
leaf:  802
leaf:  804
leaf:  827
leaf:  834
leaf:  897
leaf:  927
leaf:  932
leaf:  938
44


In [7]:
view_layout = final_layout.getLayoutProperty('viewLayout')

for e in final_layout.getEdges():
    src, tgt = graph.ends(e)
    if src.id > tgt.id:
        tmp = src
        src = tgt
        tgt = tmp
    edge_id = '{}-{}'.format(src.id, tgt.id)
    # print(edge_id, view_layout[src], view_layout[tgt])
    edges.append({
        'id': edge_id,
        'ends': (src.id, tgt.id),
        'geometry': LineString([(view_layout[src].x(), view_layout[src].y()),
                                (view_layout[tgt].x(), view_layout[tgt].y())])
    })
print(len(edges))

7991


In [8]:
bbox = tlp.computeBoundingBox(final_layout)

In [9]:
# Use the mapping function from shapely to serialize the geometry objects
for n in leaf_nodes:
    n['geometry'] = mapping(n['geometry'])
for e in edges:
    e['geometry'] = mapping(e['geometry'])
for _, n in metanodes.items():
    n['geometry'] = mapping(n['geometry'])

json_data = {
    'leaf_nodes': leaf_nodes,
    'edges': edges,
    'height': 2 if len(leaf_nodes) > 0 else 1,
    'root': root,
    'metanodes': metanodes,
    'bounding_box': [[bbox[0].x(), bbox[0].y()], [bbox[1].x(), bbox[1].y()]]
}

In [10]:
json.dump(json_data, open(myjson_filepath, 'w'), indent=2)
print('Converted to ', myjson_filepath, ' #nodes:', len(leaf_nodes), ' #edges: ', len(edges), 
      '#metanodes:', len(metanodes),
      ' height: ', json_data['height'])

Converted to  ../../../data/real-world-compiled/snap-main-comp/snap-email-eu-core-main-comp-grouseflocks-open-6.json  #nodes: 44  #edges:  7991 #metanodes: 200  height:  2
